# Lock exchange flow example
This is a test case with particles released in a simple looping flow field 
The background flow velocity is given by a lock exchange flow.

In [1]:
# Load required libraries
using Pkg
Pkg.instantiate()
Pkg.activate("..")
using Plots
using Particles
using Random
nothing

  Activating project at `j:\Master_Thesis\particles.jl`

In [2]:
randpool = MersenneTwister(0) #Generate same random numbers every time

# load useful default settings 
d=default_userdata()
n=10 #number of particles

# general setting
d["coordinates"] = "projected"
d["nparticles"]= 10 #number of particles
d["time_direction"] = :forwards # :forwards or :backwards
d["variables"] = ["x","y","z","age"] # if not stated otherwise explicitly

# load data from nc file 
dflow_map = load_nc_info(@__DIR__, r"locxx_map.nc")
# construct a grid for interpolation
# 2nd argument is number of grid points per dimension
# 3rd argument to supress spherical coordinates
interp = load_dflow_grid(dflow_map, 50, false); 

# set simulation time
# read the reftime from the file
d["reftime"] = get_reftime(dflow_map)
t_ref = get_reftime(dflow_map)
rel_times = get_times(dflow_map, t_ref)
d["dt"] = rel_times[2] - rel_times[1]
d["tstart"] = rel_times[begin]
d["tend"] = rel_times[end]

# particle settings
xnodes = dflow_map[1].vars["mesh2d_node_x"]
ynodes = dflow_map[1].vars["mesh2d_node_y"]
xmin = minimum(xnodes); xmax = maximum(xnodes);
ymin = minimum(ynodes); ymax = maximum(ynodes);
p0 = zeros(length(d["variables"]), d["nparticles"])
p0[1, :] = 0.5*(xmin+xmax)*(1 .+ rand(randpool, d["nparticles"])) 
p0[2, :] = 0.5*(ymin+ymax)*(1 .+ rand(randpool, d["nparticles"]))
d["particles"] = copy(p0)

# record grid range in each dimension
d["bbox"] = (xmin, ymin, xmax, ymax)

# plot map times 
d["plot_maps_times"] = rel_times                        # Time at which plot should be made
d["plot_maps"] = false # background flow is time evolving
                       # not suitable for using background function anymore
d["plot_maps_folder"] = "maps"
d["plot_maps_size"]=(2000,600)

# plot particle times
d["keep_particle_times"] = rel_times                        # Time at which plot should be made
d["keep_particles"] = true


┌ Info: j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc
└ @ Particles J:\Master_Thesis\particles.jl\src\dflow.jl:27


compute index:


- j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc


true

In [3]:
# prepare the interpolator for velocity field
# no need for the 3rd dimension 
u = initialize_interpolation(dflow_map, interp, "mesh2d_ucx", d["reftime"], 0.0, d["time_direction"]);
v = initialize_interpolation(dflow_map, interp, "mesh2d_ucy", d["reftime"], 0.0, d["time_direction"]);
display(typeof(u))


initialize caching for j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc mesh2d_ucx...


Particles.var"#f#25"{Interpolator, Float64, Symbol, Particles.var"#weights#24", Particles.var"#update_cache_backwards#22"{Vector{Any}}, Particles.var"#update_cache_forwards#19"{Vector{Int64}, Vector{Any}, Bool, Int64}, Vector{Any}}

initialize caching for j:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc mesh2d_ucy...


In [5]:
# explicitly prepare the grid for ploting
scale = 5
x_resolution=31 #[0,300]
y_resolution=31   #[0,1]
x = collect(range(minimum(dflow_map[1]["mesh2d_node_x"][:]), maximum(dflow_map[1]["mesh2d_node_x"][:]), length=x_resolution))
y = collect(range(minimum(dflow_map[1]["mesh2d_node_y"][:]), maximum(dflow_map[1]["mesh2d_node_y"][:]), length=y_resolution))
xs=kron(x,ones(length(y)))
ys=kron(ones(length(x)),y)

u_plot = u.(xs, ys, 0, 100)
v_plot = v.(xs, ys, 0, 100)
ar = arrow(0.03, 0.01)
p = quiver(xs, ys, quiver=(scale*u_plot, scale*v_plot), size=d["plot_maps_size"])
savefig(p, "quiver.png") 

"j:\\Master_Thesis\\particles.jl\\case_lock_exchange\\quiver.png"

In [6]:
###### Velocity function for the particles ######
function f1!(ds, s, t, i, d)
   x, y, z, age = s
   z = 0.0
   up = 0
   vp = 0
   dt = d["dt"]
   uw = u(x, y, z, t)
   vw = v(x, y, z, t)

   # The model with :
   # 1: Only flow velocities
   up = uw
   vp = vw
   # original computation for computing the particle velocity
   #(K, Kdx, Kdy) = estimate_viscosity_smag(interp, x, y, t, u, v)
   #if !(uw == vw == ua == va == 0.0)
   #   # https://doi.org/10.1016/j.ocemod.2017.11.008 eq. 27
   #   up += Kdy + randn() * sqrt(2 * K * dt) / dt
   #   vp += Kdx + randn() * sqrt(2 * K * dt) / dt
   #end
   
   epsx = 0#1
   epsz = 0#1e-5
   up += randn()*sqrt(2*epsx*dt)/dt
   vp += randn()*sqrt(2*epsz*dt)/dt

   ds.x = up
   ds.y = vp
   ds.z = 0.0
   ds.t = 1.0


   if d["time_direction"] == :backwards
      up *= -1
      vp *= -1
   end
end
d["f"]=f1!

f1! (generic function with 1 method)

In [7]:
# use streamfunction as background for plotting
function plot_vfield(d,t)
    x_resolution=41 #[0,300]
    y_resolution=21   #[0,1]
    x_min, y_min, x_max, y_max = d["bbox"]
    x = collect(range(x_min, x_max, length=x_resolution))
    y = collect(range(y_min, y_max, length=y_resolution))
    xs=kron(x,ones(length(y)))
    ys=kron(ones(length(x)),y)
    Plots.default(:size, d["plot_maps_size"])
    u_plot = u.(xs, ys, 0, 100)
    v_plot = v.(xs, ys, 0, 100)
    p = quiver(xs, ys, quiver=(scale*u_plot, scale*v_plot))
    return p
 end
 d["plot_vfield"] = plot_vfield 

plot_vfield (generic function with 1 method)

In [8]:
using Printf
# screenshot of particles
function plot_screenshot(d; dirname="", fnametag="")
    t_plot=d["keep_particle_times"];
    p=d["all_particles"]
    for i in 1:length(t_plot)
        fig = d["plot_vfield"](d, t_plot[i])
        fname = dirname* @sprintf("\\screenshot_time %.2f",t_plot[i]) *".png"
        d["plot_maps_func"](fig, d, p[i])
        savefig(fig, fname)
    end
  end

plot_screenshot (generic function with 1 method)

In [9]:
dirname = "plots"
if ~isdir(dirname)
    mkdir(dirname)
end
dirname = joinpath(@__DIR__, dirname)

"j:\\Master_Thesis\\particles.jl\\case_lock_exchange\\plots"

In [11]:
open("stdout.txt","w") do io
    redirect_stdout(io) do
        @time run_simulation(d)
        plot_screenshot(d; dirname=dirname)
    end
end